In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
# from toolbox import *
import toolbox as tb
from skeleton import *
import pandas as pd
from gensim.models import FastText
%load_ext autoreload
%autoreload 2
import os
import tqdm
from microtc.utils import tweet_iterator
import datetime
from collections import Counter
import gc
from collections import defaultdict

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
class Tokens():
    tokenized_pointer=None
    
    def tokenize(self,json_file,npy_file=None,replace=False,showProgressEach=1000):
        self.npy_file=str(os.path.splitext(json_file)[0]+".npy") if not npy_file else npy_file

        if not replace and os.path.isfile(self.npy_file):
            print(f"** Replace is off. {os.path.abspath(self.npy_file)} already exists, then load.")
        else:
            tokenized_docs=[]
            self.N=0
            print(f"** Processing {json_file} ...")
            tx = datetime.datetime.now()
            for idx, tw in enumerate(tweet_iterator(json_file)):
                tb.show_progress(showProgressEach, tx, idx)
                twTokens = tb.process_line(tw['text']) # Tokenizando tweet.
                tokenized_docs.append(twTokens)
            self.N=idx+1
        
            maxLen=len(max(tokenized_docs))
            for i, doc in enumerate(tokenized_docs):
                tokenized_docs[i]=[doc[x] if x<len(doc) else '' for x in range(maxLen)]
            
            np.save(self.npy_file,tokenized_docs)
            del(tokenized_docs)
            gc.collect()
            print(f"** Processed {self.N} lines. Saved to {os.path.abspath(self.npy_file)}.")
        
        self.pointer=np.load(self.npy_file, mmap_mode='r')
        return self
    
    def get(self,n):
        return [x for x in self.pointer[n,:] if x != '']

In [6]:
!rm -f data/*.npy

In [5]:
tokens=Tokens().tokenize("data/geo-mx-2004.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004.json ...
2020-05-12 15:15:56.669592 :: 0.00031 - Processing item #0 
2020-05-12 15:16:05.114605 :: 8.445323 - Processing item #10000 
2020-05-12 15:16:13.785040 :: 17.115758 - Processing item #20000 
2020-05-12 15:16:21.337940 :: 24.668658 - Processing item #30000 
2020-05-12 15:16:28.860174 :: 32.190892 - Processing item #40000 
2020-05-12 15:16:36.055687 :: 39.386405 - Processing item #50000 
2020-05-12 15:16:43.206188 :: 46.536906 - Processing item #60000 
2020-05-12 15:16:50.993421 :: 54.324139 - Processing item #70000 
2020-05-12 15:16:58.527787 :: 61.858505 - Processing item #80000 
2020-05-12 15:17:05.914452 :: 69.24517 - Processing item #90000 
2020-05-12 15:17:13.477884 :: 76.808602 - Processing item #100000 
2020-05-12 15:17:21.574797 :: 84.905515 - Processing item #110000 
2020-05-12 15:17:29.144386 :: 92.475104 - Processing item #120000 
2020-05-12 15:17:38.463473 :: 101.794191 - Processing item #130000 
2020-05-12 15:17:46.268061 :: 109.59877